In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

#Import fine tuned model
model_GPT2 = torch.load('ROUTE TO MODEL')
#Import tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


In [ ]:
#Function to generate a list of variation on figure of speech
def generate_figure(
    model,
    tokenizer,
    prompt,
    entry_count=1,
    entry_length=15, #maximum number of words
    top_p=0.8,
    temperature=0.5,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
              generated_list.append(output_text)
                
    return generated_list

#Function to generate multiple sentences.
def text_generation(test_data, model):
  generated_figure = []
  for i in range(10):
    x = generate_figure(model.to('cpu'), tokenizer, test_data, entry_count=1)
    generated_figure.append(x)
  return generated_figure

In [ ]:
generated_figure = text_generation("INPUT PROMPT HERE", model_GPT2)

In [ ]:
# Function to clean the generated figures
def clean_generation(generated_figure)
    counter = 1
    itemlist = []
    endproduct_list = []
    for terms in generated_figure:
        end_of_sentence = False
        termsfiltered = ''
        for term in terms:
            for char in term:
                if end_of_sentence == True:
                    break
                if((ord(char) >= 97 and ord(char) <= 122) or (ord(char) >= 65 and ord(char) <= 90)) or char in [' ', '.', ',', ':', '?', '!']:
                    termsfiltered += char
                if len(termsfiltered) >= 70 and char == ".":
                    end_of_sentence = True
        endproduct = termsfiltered.split('endoftext')
        print(counter, "=" , endproduct[0])
        print()
        endproduct_list.append(endproduct[0])
        counter += 1
    return endproduct_list
        